In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
import pandas as pd
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
df = pd.read_csv('temp_preprocess_data.csv')

In [4]:
print('shape:', df.shape)
df.head()

shape: (356251, 131)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS,BIRTH_IN_YEARS,ALL_EXT_SOURCE_MEAN,PAYMENT_RATE,INCOME_PER_PERSON,INCOME_CREDIT_IN_PERCENTAGE,ANNUITY_INCOME_IN_PERCENTAGE
0,100002,1.0,Cash loans,0,1,0,0,202500.0,406597.5,24700.5,...,1.0,0.068272,0.0,3.375,25.920548,0.161787,0.060749,202500.0,0.498036,0.121978
1,100003,0.0,Cash loans,1,1,1,0,270000.0,1293502.5,35698.5,...,0.0,0.000000,0.0,NaN,45.931507,0.466757,0.027598,135000.0,0.208736,0.132217
2,100004,0.0,Revolving loans,0,0,0,0,67500.0,135000.0,6750.0,...,0.0,0.000000,0.0,NaN,52.180822,0.642739,0.050000,67500.0,0.500000,0.100000
3,100006,0.0,Cash loans,1,1,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,52.068493,0.650442,0.094941,67500.0,0.431748,0.219900
4,100007,0.0,Cash loans,0,1,0,0,121500.0,513000.0,21865.5,...,0.0,0.000000,0.0,NaN,54.608219,0.322738,0.042623,121500.0,0.236842,0.179963


In [5]:
train_df = df.loc[df['TARGET'].notnull(), :]
test_df = df.loc[df['TARGET'].isnull(), :]
test_df.drop(columns= ['TARGET'], axis=1, inplace=True)

train_df_enc = pd.get_dummies(train_df)
test_df_enc = pd.get_dummies(test_df)
print('train shape:', train_df_enc.shape) 
print('test shape:', test_df_enc.shape) 

/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


train shape: (307507, 251)
test shape: (48744, 248)


In [6]:
train_df_enc_cols = set(train_df_enc.columns)
test_df_enc_cols = set(test_df_enc.columns)

print(train_df_enc_cols.difference(test_df_enc_cols))

{'NAME_FAMILY_STATUS_Unknown', 'TARGET', 'NAME_INCOME_TYPE_Maternity leave'}


In [7]:
y = train_df_enc['TARGET']
## align dataframes 
X, X_test = train_df_enc.align(test_df_enc, join='inner', axis=1)

In [8]:
X.shape, X_test.shape, y.shape

((307507, 248), (48744, 248), (307507,))

In [9]:

imp = SimpleImputer(missing_values=np.nan, strategy='median')

In [10]:
Test_ids = X_test['SK_ID_CURR']
X.drop(columns= ['SK_ID_CURR'], inplace=True)
X_test.drop(columns= ['SK_ID_CURR'], inplace=True)

In [11]:
X.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0,1,0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,0,1,0,1,0
1,1,1,1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,0,0,0,0,0


In [12]:
## fix the inf problem 
X['AMT_CREDIT_DEBT_RATIO'] = X['AMT_CREDIT_DEBT_RATIO'].apply(lambda x: x if ~np.isinf(x) else 0)
X_test['AMT_CREDIT_DEBT_RATIO'] = X_test['AMT_CREDIT_DEBT_RATIO'].apply(lambda x: x if ~np.isinf(x) else 0)

In [13]:
## impute missing values by its median 
imp.fit(X)
X = imp.transform(X)
X_test = imp.fit_transform(X_test)

# Baseline: No sampling
- expecting model to overfit 

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= 42, stratify=y)
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_val = rf.predict(X_val)
print('classifaction report on training set')
print(classification_report(y_train, y_pred_train, labels=[0,1]))
print('--------------------------------------------------------------')
print('classifaction report on validation set')
print(classification_report(y_val, y_pred_val, labels=[0,1]))

classifaction report on training set
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    226145
           1       1.00      1.00      1.00     19860

    accuracy                           1.00    246005
   macro avg       1.00      1.00      1.00    246005
weighted avg       1.00      1.00      1.00    246005

--------------------------------------------------------------
classifaction report on validation set
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56537
           1       0.76      0.00      0.01      4965

    accuracy                           0.92     61502
   macro avg       0.84      0.50      0.48     61502
weighted avg       0.91      0.92      0.88     61502



We know the model above is garbage - since the the the dummy classifier (i.e i==0) will score 96% accuracy; therefore the notion accuracy in this sense is meaningless. Our objective is to increase the f1-score for both classes. Ideally we want the f1-score for both classes to be close to 1 then we can use the accuracy score evaluate the performance of the model. We're going to attempt different sampling techniques to achieve our goal. 

# Sampling methods

In [15]:
# create another df for sampling 
diff = train_df_enc_cols.difference(test_df_enc_cols)
cols = [c for c in train_df_enc.columns if c not in diff]

df_prime = pd.DataFrame(X, columns = cols[1:])
# add target
df_prime['TARGET'] = y

# separate classes
df_class0 = df_prime.loc[df_prime['TARGET']==0, :]
df_class1 = df_prime.loc[df_prime['TARGET']==1, :]


## down-sampling

In [16]:
print('Before sampling')
print(df_class0.shape, df_class1.shape)

df_class0_down_sample = df_class0.sample(df_class1.shape[0])

print('After sampling')
print(df_class0_down_sample.shape, df_class1.shape)

# concat the two classes (down sample)
df_ds = pd.concat([df_class0_down_sample, df_class1], axis = 0)
print('new df shape:', df_ds.shape)

Before sampling
(282682, 248) (24825, 248)
After sampling
(24825, 248) (24825, 248)
new df shape: (49650, 248)


In [17]:
df_ds.TARGET.value_counts()

1.0    24825
0.0    24825
Name: TARGET, dtype: int64

In [18]:
X = df_ds.drop(columns = ['TARGET'], axis=1)
y = df_ds['TARGET']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= 42, stratify=y)

### random forest

In [19]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_val = rf.predict(X_val)
print('classifaction report on training set')
print(classification_report(y_train, y_pred_train, labels=[0,1]))
print('--------------------------------------------------------------')
print('classifaction report on validation set')
print(classification_report(y_val, y_pred_val, labels=[0,1]))

classifaction report on training set
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19860
           1       1.00      1.00      1.00     19860

    accuracy                           1.00     39720
   macro avg       1.00      1.00      1.00     39720
weighted avg       1.00      1.00      1.00     39720

--------------------------------------------------------------
classifaction report on validation set
              precision    recall  f1-score   support

           0       0.68      0.69      0.69      4965
           1       0.69      0.68      0.68      4965

    accuracy                           0.68      9930
   macro avg       0.68      0.68      0.68      9930
weighted avg       0.68      0.68      0.68      9930



In [20]:
rf.feature_importances_.shape

(247,)

In [21]:
features = cols[1:]
fm = {'feature': features,
    'importance': rf.feature_importances_}
feature_importance = pd.DataFrame(fm)
feature_importance.sort_values(by='importance', ascending=False).head(10)

,feature,importance
111,ALL_EXT_SOURCE_MEAN,0.062996
31,EXT_SOURCE_2,0.045729
32,EXT_SOURCE_3,0.042126
107,AMT_CREDIT_DEBT_RATIO,0.026097
112,PAYMENT_RATE,0.026005
10,DAYS_EMPLOYED,0.025586
9,DAYS_BIRTH,0.024729
110,BIRTH_IN_YEARS,0.024138
12,DAYS_ID_PUBLISH,0.024074
11,DAYS_REGISTRATION,0.023583


### logistic regression

In [22]:
## logistic regression
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)
X_std = scaler.transform(X)
X_test_std = scaler.fit_transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(X_std, y, test_size=0.2, random_state= 42, stratify=y)
logreg = LogisticRegression(solver='lbfgs', max_iter =1000, random_state=42)
logreg.fit(X_train, y_train)

y_pred_train = logreg.predict(X_train)
y_pred_val = logreg.predict(X_val)
print('classifaction report on training set')
print(classification_report(y_train, y_pred_train, labels=[0,1]))
print('--------------------------------------------------------------')
print('classifaction report on validation set')
print(classification_report(y_val, y_pred_val, labels=[0,1]))

classifaction report on training set
              precision    recall  f1-score   support

           0       0.68      0.68      0.68     19860
           1       0.68      0.68      0.68     19860

    accuracy                           0.68     39720
   macro avg       0.68      0.68      0.68     39720
weighted avg       0.68      0.68      0.68     39720

--------------------------------------------------------------
classifaction report on validation set
              precision    recall  f1-score   support

           0       0.69      0.68      0.68      4965
           1       0.68      0.69      0.69      4965

    accuracy                           0.69      9930
   macro avg       0.69      0.69      0.69      9930
weighted avg       0.69      0.69      0.69      9930



In [23]:
logreg.coef_.shape

(1, 247)

In [24]:
#importance = model.coef_
features = cols[1:]
fm = {'feature': features,
    'importance':  logreg.coef_.reshape((247,))}
feature_importance = pd.DataFrame(fm)
feature_importance.sort_values(by='importance', ascending=False).head(20)

,feature,importance
5,AMT_CREDIT,0.591735
37,COMMONAREA_AVG,0.444095
62,BASEMENTAREA_MEDI,0.386023
43,LIVINGAPARTMENTS_AVG,0.331523
59,NONLIVINGAPARTMENTS_MODE,0.247886
4,AMT_INCOME_TOTAL,0.240527
35,YEARS_BEGINEXPLUATATION_AVG,0.220793
6,AMT_ANNUITY,0.207400
76,OBS_30_CNT_SOCIAL_CIRCLE,0.205845
44,LIVINGAREA_AVG,0.193959


## up-sampling 

In [25]:
print('Before sampling')
print(df_class0.shape, df_class1.shape)

df_class1_up_sample = df_class1.sample(df_class0.shape[0], replace=True)

print('After sampling')
print(df_class0.shape, df_class1_up_sample.shape)

# concat the two classes (down sample)
df_us = pd.concat([df_class0, df_class1_up_sample], axis = 0)
print('new df shape:', df_us.shape)

Before sampling
(282682, 248) (24825, 248)
After sampling
(282682, 248) (282682, 248)
new df shape: (565364, 248)


In [26]:
df_us.TARGET.value_counts()

1.0    282682
0.0    282682
Name: TARGET, dtype: int64

In [27]:
X = df_us.drop(columns = ['TARGET'], axis=1)
y = df_us['TARGET']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= 42, stratify=y)

### random forest

In [28]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_val = rf.predict(X_val)
print('classifaction report on training set')
print(classification_report(y_train, y_pred_train, labels=[0,1]))
print('--------------------------------------------------------------')
print('classifaction report on validation set')
print(classification_report(y_val, y_pred_val, labels=[0,1]))

classifaction report on training set
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    226145
           1       1.00      1.00      1.00    226146

    accuracy                           1.00    452291
   macro avg       1.00      1.00      1.00    452291
weighted avg       1.00      1.00      1.00    452291

--------------------------------------------------------------
classifaction report on validation set
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56537
           1       0.99      1.00      1.00     56536

    accuracy                           1.00    113073
   macro avg       1.00      1.00      1.00    113073
weighted avg       1.00      1.00      1.00    113073



This doesn't look good ... over-fitting 

In [30]:
rf_test_pred = rf.predict_proba(X_test)[:,1]

submission3_dict = {'SK_ID_CURR': Test_ids, 
            'TARGET': rf_test_pred }
rf_submission3 = pd.DataFrame(submission3_dict)
rf_submission3.head()

,SK_ID_CURR,TARGET
307507,100001,0.06
307508,100005,0.15
307509,100013,0.05
307510,100028,0.10
307511,100038,0.27


In [31]:
#rf_submission3.to_csv('submission3.csv', index=False) ## 73% accuracy

In [32]:
features = cols[1:]
fm = {'feature': features,
    'importance': rf.feature_importances_}
feature_importance = pd.DataFrame(fm)
feature_importance.sort_values(by='importance', ascending=False).head(10)

,feature,importance
111,ALL_EXT_SOURCE_MEAN,0.069198
31,EXT_SOURCE_2,0.045948
32,EXT_SOURCE_3,0.045820
112,PAYMENT_RATE,0.027758
10,DAYS_EMPLOYED,0.026476
9,DAYS_BIRTH,0.026189
107,AMT_CREDIT_DEBT_RATIO,0.025399
110,BIRTH_IN_YEARS,0.025036
12,DAYS_ID_PUBLISH,0.024598
11,DAYS_REGISTRATION,0.023515


### logistic regression 

In [33]:
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)
X_std = scaler.transform(X)
X_test_std = scaler.fit_transform(X_test)

X_train_std, X_val_std, y_train, y_val = train_test_split(X_std, y, test_size=0.2, random_state= 42, stratify=y)
logreg = LogisticRegression(solver='lbfgs', max_iter =1000, random_state=42)
logreg.fit(X_train_std, y_train)

y_pred_train = logreg.predict(X_train_std)
y_pred_val = logreg.predict(X_val_std)
print('classifaction report on training set')
print(classification_report(y_train, y_pred_train, labels=[0,1]))
print('--------------------------------------------------------------')
print('classifaction report on validation set')
print(classification_report(y_val, y_pred_val, labels=[0,1]))

classifaction report on training set
              precision    recall  f1-score   support

           0       0.69      0.69      0.69    226145
           1       0.69      0.68      0.69    226146

    accuracy                           0.69    452291
   macro avg       0.69      0.69      0.69    452291
weighted avg       0.69      0.69      0.69    452291

--------------------------------------------------------------
classifaction report on validation set
              precision    recall  f1-score   support

           0       0.69      0.70      0.69     56537
           1       0.69      0.69      0.69     56536

    accuracy                           0.69    113073
   macro avg       0.69      0.69      0.69    113073
weighted avg       0.69      0.69      0.69    113073



# SMOTE

In [34]:
#! pip install imbalanced-learn

In [35]:
#! pip install delayed

In [36]:
from imblearn.over_sampling import SMOTE

In [44]:
X = df_prime.drop(columns = ['TARGET'], axis=1)
y = df_prime['TARGET']
X.shape, y.shape

((307507, 247), (307507,))

In [45]:
y.value_counts()

0.0    282682
1.0     24825
Name: TARGET, dtype: int64

In [49]:
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X,y)

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= 42, stratify=y)

In [50]:
y_sm.value_counts()

1.0    282682
0.0    282682
Name: TARGET, dtype: int64

In [52]:
X_sm.shape

(565364, 247)

In [54]:
y_sm.shape

(565364,)

In [56]:
X_train, X_val, y_train, y_val = train_test_split(X_sm, y_sm, test_size=0.2, random_state= 42, stratify=y_sm)
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_val = rf.predict(X_val)
print('classifaction report on training set')
print(classification_report(y_train, y_pred_train, labels=[0,1]))
print('--------------------------------------------------------------')
print('classifaction report on validation set')
print(classification_report(y_val, y_pred_val, labels=[0,1]))

classifaction report on training set
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    226145
           1       1.00      1.00      1.00    226146

    accuracy                           1.00    452291
   macro avg       1.00      1.00      1.00    452291
weighted avg       1.00      1.00      1.00    452291

--------------------------------------------------------------
classifaction report on validation set
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56537
           1       1.00      0.91      0.95     56536

    accuracy                           0.96    113073
   macro avg       0.96      0.96      0.96    113073
weighted avg       0.96      0.96      0.96    113073



In [57]:
rf_test_pred = rf.predict_proba(X_test)[:,1]

submission4_dict = {'SK_ID_CURR': Test_ids, 
            'TARGET': rf_test_pred }
rf_submission4 = pd.DataFrame(submission4_dict)
#rf_submission4.to_csv('submission4.csv', index=False) accuaracy 64%
rf_submission4.head()

,SK_ID_CURR,TARGET
307507,100001,0.06
307508,100005,0.50
307509,100013,0.27
307510,100028,0.12
307511,100038,0.44
